# Import and Read data


In [1]:
import sys
sys.path.append('.')  # Add current directory to path
from build_graph_and_train import *

/opt/conda/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
partition = 200

In [3]:
df = pd.read_csv(f"../../../../data/top30groups/DNDF_OneHotLongLatCombined/combined/combined{partition}.csv")

In [4]:
import os 
if not os.path.isdir(f"Results{partition}"):
    os.mkdir(f"Results{partition}")

# Create longlat feature

In [5]:
geodata = ['longitude', 'latitude']
combined_geo = df.copy()
combined_geo['longlat'] = list(zip(df['longitude'], df['latitude']))
combined_geo = combined_geo.drop(columns=geodata)

In [6]:
import ast

def to_tuple_if_needed(val):
    if isinstance(val, str):
        return ast.literal_eval(val)
    return val  # already a tuple

combined_geo['longlat'] = combined_geo['longlat'].apply(to_tuple_if_needed)

In [7]:
combined_geo.dtypes

extended         int64
vicinity         int64
multiple       float64
success          int64
suicide          int64
attacktype1      int64
targtype1        int64
target1          int64
individual       int64
weaptype1        int64
nkill          float64
property         int64
ishostkid      float64
gname           object
longlat         object
dtype: object

# Weapon type prediction

In [8]:
label_index = {g: i for i, g in enumerate(sorted(df['gname'].unique()))}
continuous_col = 'weaptype1'
data, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, row_to_node_index = build_graph_data(combined_geo, label_index, continuous_col=continuous_col)
args = {
    'partition': f"gtd{partition}",
    'embed_dim': 16,
    'lr': 0.01,
    'epochs': 1500,
    'feat_dropout': 0,
    'n_tree': 80,
    'tree_depth': 10,
    'tree_feature_rate': 0.5,
    'n_class': len(label_index)  
}
best_acc, best_epoch = train_joint(data, data.edge_index, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, args, row_to_node_index)

with open(f"Results{partition}/Results_{continuous_col}_prediction", "w") as f:
    f.write(f"Best acc: {best_acc} in epoch {best_epoch} for {continuous_col} prediction")

hej


/home/jovyan/buggy_GTD/GTD_2025/Codes/Baselines/graphrfi/GraphRfi_OneHotLongLat/build_graph_and_train.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_nrf = torch.tensor(y_nrf, dtype=torch.long).to(device)


Epoch 01 | GCN MSE Loss: 394.4083 | NRF Loss: 3.4010 | JOINT Loss: 397.8093 | NRF Acc: 0.0372
Epoch 02 | GCN MSE Loss: 264.6898 | NRF Loss: 3.3965 | JOINT Loss: 268.0862 | NRF Acc: 0.0220
Epoch 03 | GCN MSE Loss: 168.4504 | NRF Loss: 3.3964 | JOINT Loss: 171.8468 | NRF Acc: 0.0220
Epoch 04 | GCN MSE Loss: 103.0217 | NRF Loss: 3.3996 | JOINT Loss: 106.4213 | NRF Acc: 0.0220
Epoch 05 | GCN MSE Loss: 64.3697 | NRF Loss: 3.4018 | JOINT Loss: 67.7716 | NRF Acc: 0.0220
Epoch 06 | GCN MSE Loss: 47.2893 | NRF Loss: 3.4017 | JOINT Loss: 50.6911 | NRF Acc: 0.0220
Epoch 07 | GCN MSE Loss: 46.2348 | NRF Loss: 3.3996 | JOINT Loss: 49.6345 | NRF Acc: 0.0220
Epoch 08 | GCN MSE Loss: 54.7830 | NRF Loss: 3.4011 | JOINT Loss: 58.1841 | NRF Acc: 0.0220
Epoch 09 | GCN MSE Loss: 66.5139 | NRF Loss: 3.3967 | JOINT Loss: 69.9106 | NRF Acc: 0.0220
Epoch 10 | GCN MSE Loss: 76.2517 | NRF Loss: 3.3921 | JOINT Loss: 79.6438 | NRF Acc: 0.0220
Epoch 11 | GCN MSE Loss: 81.0127 | NRF Loss: 3.3917 | JOINT Loss: 84.404

# Fatality rate


In [9]:
label_index = {g: i for i, g in enumerate(sorted(df['gname'].unique()))}
continuous_col = 'nkill'
data, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, row_to_node_index = build_graph_data(combined_geo, label_index, continuous_col=continuous_col)
args = {
    'partition': f"gtd{partition}",
    'embed_dim': 16,
    'lr': 0.01,
    'epochs': 1500,
    'feat_dropout': 0,
    'n_tree': 80,
    'tree_depth': 10,
    'tree_feature_rate': 0.5,
    'n_class': len(label_index)  
}
best_acc, best_epoch = train_joint(data, data.edge_index, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, args, row_to_node_index)

with open(f"Results{partition}/Results_{continuous_col}_prediction", "w") as f:
    f.write(f"Best acc: {best_acc} in epoch {best_epoch} for {continuous_col} prediction")

hej


/home/jovyan/buggy_GTD/GTD_2025/Codes/Baselines/graphrfi/GraphRfi_OneHotLongLat/build_graph_and_train.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_nrf = torch.tensor(y_nrf, dtype=torch.long).to(device)


Epoch 01 | GCN MSE Loss: 209.0026 | NRF Loss: 3.3998 | JOINT Loss: 212.4024 | NRF Acc: 0.0426
Epoch 02 | GCN MSE Loss: 177.9814 | NRF Loss: 3.4000 | JOINT Loss: 181.3814 | NRF Acc: 0.0305
Epoch 03 | GCN MSE Loss: 170.0471 | NRF Loss: 3.4018 | JOINT Loss: 173.4490 | NRF Acc: 0.0343
Epoch 04 | GCN MSE Loss: 172.4350 | NRF Loss: 3.4026 | JOINT Loss: 175.8376 | NRF Acc: 0.0437
Epoch 05 | GCN MSE Loss: 171.0300 | NRF Loss: 3.4015 | JOINT Loss: 174.4315 | NRF Acc: 0.0423
Epoch 06 | GCN MSE Loss: 164.3434 | NRF Loss: 3.3990 | JOINT Loss: 167.7424 | NRF Acc: 0.0442
Epoch 07 | GCN MSE Loss: 156.0691 | NRF Loss: 3.3992 | JOINT Loss: 159.4683 | NRF Acc: 0.0442
Epoch 08 | GCN MSE Loss: 149.4823 | NRF Loss: 3.3975 | JOINT Loss: 152.8798 | NRF Acc: 0.0439
Epoch 09 | GCN MSE Loss: 146.1793 | NRF Loss: 3.3954 | JOINT Loss: 149.5747 | NRF Acc: 0.0450
Epoch 10 | GCN MSE Loss: 145.6264 | NRF Loss: 3.3927 | JOINT Loss: 149.0191 | NRF Acc: 0.0479
Epoch 11 | GCN MSE Loss: 145.9581 | NRF Loss: 3.3906 | JOINT

# Target Type

In [10]:
label_index = {g: i for i, g in enumerate(sorted(df['gname'].unique()))}
continuous_col = 'targtype1'
data, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, row_to_node_index = build_graph_data(combined_geo, label_index, continuous_col=continuous_col)
args = {
    'partition': f"gtd{partition}",
    'embed_dim': 16,
    'lr': 0.01,
    'epochs': 1500,
    'feat_dropout': 0,
    'n_tree': 80,
    'tree_depth': 10,
    'tree_feature_rate': 0.5,
    'n_class': len(label_index)  
}
best_acc, best_epoch = train_joint(data, data.edge_index, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, args, row_to_node_index)

with open(f"Results{partition}/Results_{continuous_col}_prediction", "w") as f:
    f.write(f"Best acc: {best_acc} in epoch {best_epoch} for {continuous_col} prediction")

hej


/home/jovyan/buggy_GTD/GTD_2025/Codes/Baselines/graphrfi/GraphRfi_OneHotLongLat/build_graph_and_train.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_nrf = torch.tensor(y_nrf, dtype=torch.long).to(device)


Epoch 01 | GCN MSE Loss: 352.0916 | NRF Loss: 3.3986 | JOINT Loss: 355.4903 | NRF Acc: 0.0509
Epoch 02 | GCN MSE Loss: 238.4293 | NRF Loss: 3.3954 | JOINT Loss: 241.8247 | NRF Acc: 0.0426
Epoch 03 | GCN MSE Loss: 157.2313 | NRF Loss: 3.3930 | JOINT Loss: 160.6243 | NRF Acc: 0.0351
Epoch 04 | GCN MSE Loss: 106.0629 | NRF Loss: 3.3886 | JOINT Loss: 109.4515 | NRF Acc: 0.0570
Epoch 05 | GCN MSE Loss: 80.1030 | NRF Loss: 3.3924 | JOINT Loss: 83.4955 | NRF Acc: 0.0520
Epoch 06 | GCN MSE Loss: 73.3844 | NRF Loss: 3.3905 | JOINT Loss: 76.7749 | NRF Acc: 0.0520
Epoch 07 | GCN MSE Loss: 79.5355 | NRF Loss: 3.3897 | JOINT Loss: 82.9252 | NRF Acc: 0.0503
Epoch 08 | GCN MSE Loss: 91.5863 | NRF Loss: 3.3896 | JOINT Loss: 94.9759 | NRF Acc: 0.0509
Epoch 09 | GCN MSE Loss: 103.0923 | NRF Loss: 3.3878 | JOINT Loss: 106.4800 | NRF Acc: 0.0530
Epoch 10 | GCN MSE Loss: 109.9332 | NRF Loss: 3.3869 | JOINT Loss: 113.3201 | NRF Acc: 0.0520
Epoch 11 | GCN MSE Loss: 110.8442 | NRF Loss: 3.3849 | JOINT Loss: 1

# Attack type

In [12]:
label_index = {g: i for i, g in enumerate(sorted(df['gname'].unique()))}
continuous_col = 'attacktype1'
data, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, row_to_node_index = build_graph_data(combined_geo, label_index, continuous_col=continuous_col)
args = {
    'partition': f"gtd{partition}",
    'embed_dim': 16,
    'lr': 0.01,
    'epochs': 1500,
    'feat_dropout': 0,
    'n_tree': 80,
    'tree_depth': 10,
    'tree_feature_rate': 0.5,
    'n_class': len(label_index)  
}
best_acc, best_epoch = train_joint(data, data.edge_index, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, args, row_to_node_index)

with open(f"Results{partition}/Results_{continuous_col}_prediction", "w") as f:
    f.write(f"Best acc: {best_acc} in epoch {best_epoch} for {continuous_col} prediction")

hej


/home/jovyan/buggy_GTD/GTD_2025/Codes/Baselines/graphrfi/GraphRfi_OneHotLongLat/build_graph_and_train.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_nrf = torch.tensor(y_nrf, dtype=torch.long).to(device)


Epoch 01 | GCN MSE Loss: 1256.5626 | NRF Loss: 3.4004 | JOINT Loss: 1259.9630 | NRF Acc: 0.0126
Epoch 02 | GCN MSE Loss: 1076.6874 | NRF Loss: 3.3929 | JOINT Loss: 1080.0802 | NRF Acc: 0.0635
Epoch 03 | GCN MSE Loss: 918.3909 | NRF Loss: 3.3946 | JOINT Loss: 921.7855 | NRF Acc: 0.0335
Epoch 04 | GCN MSE Loss: 775.4100 | NRF Loss: 3.3890 | JOINT Loss: 778.7990 | NRF Acc: 0.0281
Epoch 05 | GCN MSE Loss: 649.9444 | NRF Loss: 3.3898 | JOINT Loss: 653.3342 | NRF Acc: 0.0284
Epoch 06 | GCN MSE Loss: 538.1488 | NRF Loss: 3.3890 | JOINT Loss: 541.5378 | NRF Acc: 0.0284
Epoch 07 | GCN MSE Loss: 439.5701 | NRF Loss: 3.3846 | JOINT Loss: 442.9547 | NRF Acc: 0.0295
Epoch 08 | GCN MSE Loss: 353.8224 | NRF Loss: 3.3847 | JOINT Loss: 357.2072 | NRF Acc: 0.0276
Epoch 09 | GCN MSE Loss: 280.1245 | NRF Loss: 3.3820 | JOINT Loss: 283.5065 | NRF Acc: 0.0252
Epoch 10 | GCN MSE Loss: 217.5524 | NRF Loss: 3.3836 | JOINT Loss: 220.9359 | NRF Acc: 0.0244
Epoch 11 | GCN MSE Loss: 165.2701 | NRF Loss: 3.3880 | J